In [ ]:
# instalar librerias

!pip install torch==2.2.2
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q bitsandbytes accelerate xformers einops
!pip install -q hf_transfer
!pip install trl
!pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s eta 0:00:00
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-man

In [ ]:
# importar librerias
#import torch
# from huggingface_hub import notebook_login
import torch
import transformers
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
#from trl import SFTTrainer
#from unsloth import FastLanguageModel


In [ ]:
model_name = 'meta-llama/Meta-Llama-3-8B'
access_token = ''

# SFT PARAMETERS
max_seq_length = 40
packing = True
device_map = 'auto'

# BITSANDBYTES PARAMETERS
use_4bit = True
bnb_4bit_compute_dtype = 'float16'
bnb_4bit_quant_type = 'nf4'
use_nested_quant = False

# QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit = use_4bit,
    bnb_4bit_quant_type = bnb_4bit_quant_type,
    bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant = use_nested_quant,
)

# check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
  major, _ = torch.cuda.get_device_capability()
  if major >= 8:
    print('Setting BF16 to True')
    bf16 = True
  else:
    bf16 = False

In [ ]:
# load model and tokenizer

model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config = bnb_config,
                                             device_map = 'auto',
                                             token = access_token)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    token = access_token,
    trust_remote_code = True
    )

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_size = 'right'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
###########################################
########## LORA PARAMETERS ################
###########################################
lora_alpha = 128
lora_r = 64
lora_dropout = 0.1

# load LoRA configuration
from peft import LoraConfig

lora_config = LoraConfig(
    lora_alpha = lora_alpha,
    lora_dropout = lora_dropout,
    r = lora_r,
    bias = 'none'
)

# set training parameters

# TRAININGARGUMENTS PARAMETERS
output_dir = './results'
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = 'paged_adamw_32bit'
lr_scheduler_type = 'constant'
#max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 25

training_arguments = TrainingArguments(
    output_dir = output_dir,
    num_train_epochs = num_train_epochs,
    per_device_train_batch_size = per_device_train_batch_size,
    gradient_accumulation_steps = gradient_accumulation_steps,
    optim = optim,
    save_steps = save_steps,
    logging_steps = logging_steps,
    learning_rate = learning_rate,
    weight_decay = weight_decay,
    fp16 = fp16,
    bf16 = bf16,
    max_grad_norm = max_grad_norm,
    warmup_ratio = warmup_ratio,
    group_by_length = group_by_length,
    lr_scheduler_type = lr_scheduler_type,
    report_to = 'tensorboard'
)

In [ ]:
# load data and metrics
import pandas as pd
from sklearn.model_selection import train_test_split
from trl import SFTTrainer
import datasets
from datasets import Dataset

data = pd.read_csv('training_test_data.csv')

In [ ]:
# cross validation

import numpy as np
from sklearn.model_selection import KFold

n_splits = 4
kf = KFold(n_splits=n_splits, shuffle=False) # create folds

real_phrases = []
generated_phrases = []
lemmas = []

for train_index, test_index in kf.split(data):
    training_data, test_data = data['text'][train_index], data['text'][test_index]

    training_data = Dataset.from_pandas(training_data.to_frame().reset_index())
    test_data = Dataset.from_pandas(test_data.to_frame().reset_index())

    trainer = SFTTrainer(
      model = model,
      train_dataset = training_data,
      peft_config = lora_config,
      dataset_text_field = 'text',
      max_seq_length = max_seq_length,
      tokenizer = tokenizer,
      args = training_arguments,
      packing = packing
    )

    trainer.train()

    for p in test_data:
      text = p['text'].split('Phrase: ')
      prompt = text[0]
      ph = text[1]
      real_phrases.append(ph)
      lemmas.append(prompt.split('Lemmas: ')[1].strip('\n'))

      inputs = tokenizer(prompt, return_tensors = 'pt')
      generate_ids = model.generate(inputs.input_ids, max_length = 50)
      output = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

      try:
        generated_phrases.append(output.split('Phrase: ')[1])
      except:
        generated_phrases.append('')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length, packing. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1960: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:181: UserWarning: You passed a `packing` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1960: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. 

Step,Training Loss
25,2.023200


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66644c1f-1db4352f39407e3a2717d035;d9a94db6-fa35-42eb-812a-1cef2d036bad)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66644c4b-75a1623e60c9463377627fb7;c366a67e-6c9d-4dd9-9b79-69

Step,Training Loss
25,2.039500


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66644d0f-1c98719b32248c2a6ccfc1b4;d91b604c-aad2-4749-97c8-09f34508247f)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66644d46-0cd88d6a147429023902d783;82ae3918-a56f-4d4c-95a2-95

Step,Training Loss
25,2.052400


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66644e15-21fd26c14b0efdc75bce87a9;2fbaf258-32fe-4246-824c-a775c1614f95)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66644e41-0a7d685b6fbfb53c22a3fa70;f70b1440-e6e5-4036-ab8b-4b

Step,Training Loss
25,2.002600


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66644f06-242e1ef2142bb82972851bf1;afad38b1-22f6-4002-b257-32dd6fb796de)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3-8B/resolve/main/config.json.
Access to model meta-llama/Meta-Llama-3-8B is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in meta-llama/Meta-Llama-3-8B.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in meta-llama/Meta-Llama-3-8B - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:611: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66644f33-3800be74302899df71ec3618;a219855f-5616-4d4f-8306-29

In [ ]:
from bs4 import BeautifulSoup

clean = []

for i in generated_phrases:
  clean_text = BeautifulSoup(i, "lxml").text.rstrip()
  clean.append(clean_text)

results = pd.DataFrame()
results['lemmas'] = lemmas
results['real phrases'] = real_phrases
results['generated phrases'] = clean

results.to_csv(f'llama3_results_alpha_{lora_alpha}_r_{lora_r}_n{n_splits}.csv')

In [ ]:
for i in generated_phrases:
  print(i)

El niño escribe en la ventana con el dedo.
El lobo se teñía con harina en las patas.
La pata es negra y peluda.
El gato come a ratatita.
Laura y Malena son gemelos.
Me ves bien.
La niña compra ropa en la tienda de ropa.
La gallina pone el huevo de oro.
La chica nada en la piscina con cubierta.
Me gusta la pandereta.
La mamá cabra tiene siete cabritillos.
Nora se puso morena.
El lobo va al molino.
Yo me gusta los churros rellenos.
Yo duermo la noche.
Ese señor va a pasear al pinar.
Yo duermo en la cama.
Los siete cabritillos juegan contentos.
La arpa produce diamantes.
La niña aplaude con las manos.
El cerdito corre rápido.
El lobo quiere comerlo.
Los niños juegan en el parque.
Luis está un poco loco.
Vemos un topillo en otoño.
La chica merienda un bocadillo de chorizo con leche.
La rosa es rubia.
La niña está sentada en la roca.
El ruiseñor canta en la rama.
Los niños juegan al corro.
Los chicos acaban con el borrico.
La mamá lee a la niña.
Todo los niños se asustan.
Pili ha metido un 